In [11]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
import statsmodels.formula.api as smf
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

In [12]:
table = pd.read_excel('Avon_Data.xlsx')
table.columns

Index([u'cv__Campaign__r.Name', u'Year', u'City', u'Walk_Participation',
       u'Walk_Status', u'Classification', u'FUL_CHECKED',
       u'cv__Registration_Date__c', u'Withdrew', u'Active',
       u'Registration_Days_Out', u'Opportunity_Amount', u'AF_PROMO__c',
       u'AF_REG_WAY_OF_ENTRY__c', u'AF_REG_SOURCE__c',
       u'AF_REG_SOURCE_DETAIL__c', u'AF_LEAD_GENERATED_DATE__c',
       u'AF_LEAD_SOURCE__c', u'AF_LEAD_SUBSOURCE__c', u'Age_During_Event__c',
       u'cv__Team__r.Name', u'Total_Raised_Master'],
      dtype='object')

In [13]:
table_df = pd.DataFrame(table)

In [14]:
filter_data=table_df[table_df['Year']!= 2009]
filter_data=filter_data[table_df['Year']!= 2017]

C:\Users\rvardhineni\AppData\Local\Continuum\Anaconda2\envs\py27\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [15]:
filter_data

,cv__Campaign__r.Name,Year,City,Walk_Participation,Walk_Status,Classification,FUL_CHECKED,cv__Registration_Date__c,Withdrew,Active,...,AF_PROMO__c,AF_REG_WAY_OF_ENTRY__c,AF_REG_SOURCE__c,AF_REG_SOURCE_DETAIL__c,AF_LEAD_GENERATED_DATE__c,AF_LEAD_SOURCE__c,AF_LEAD_SUBSOURCE__c,Age_During_Event__c,cv__Team__r.Name,Total_Raised_Master
1,2010 Boston,2010,Boston,Crew,Active,Crew,1,2010-01-04,0,1,...,NaN,Web Site,Unknown,NaN,NaT,Other,NaN,23.0,Cure-vaceous Cuties,1695.0
2,2010 Boston,2010,Boston,Crew,Active,Crew,1,2010-02-12,0,1,...,NaN,Phone/Local Office,Unknown,NaN,2009-10-05,Email,NaN,57.0,NaN,500.0
3,2010 Boston,2010,Boston,Crew,Active,Crew,1,2010-01-03,0,1,...,NaN,Web Site,Email,NaN,NaT,Email,NaN,NaN,NaN,0.0
4,2010 Boston,2010,Boston,Crew,Active,Crew,1,2010-01-02,0,1,...,NaN,Web Site,Email,NaN,NaT,Email,NaN,48.0,NaN,0.0
5,2010 Boston,2010,Boston,Crew,Active,Crew,1,2010-01-06,0,1,...,NaN,Web Site,Outreach,NaN,NaT,Outreach,NaN,43.0,NaN,0.0
6,2010 Boston,2010,Boston,Crew,Active,Crew,1,2010-02-15,0,1,...,NaN,Phone/Local Office,Unknown,NaN,2010-01-06,Internet,Internet: Search Engine,22.0,NaN,190.0
7,2010 Boston,2010,Boston,Crew,Active,Crew,0,2010-01-07,0,1,...,NaN,Web Site,TV,Cable: Food Network,NaT,TV,TV: CABLE Food Network,NaN,NaN,0.0
8,2010 Boston,2010,Boston,Crew,Active,Crew,1,2010-01-09,0,1,...,NaN,Web Site,Email,NaN,NaT,Email,NaN,37.0,NaN,0.0
9,2010 Boston,2010,Boston,Crew,Active,Crew,1,2010-01-13,0,1,...,IM2010,Phone/Local Office,Friend/Family/Acquaintance,NaN,2010-01-09,Friend/Family/Acquaintance,NaN,58.0,Fran's Clan,975.0
10,2010 Boston,2010,Boston,Crew,Active,Crew,1,2010-01-13,0,1,...,IM2010,Phone/Local Office,Outreach,NaN,2010-01-11,Outreach,NaN,62.0,Fran's Clan,575.0


In [16]:
unknown_values = ['Unknown', 'nan','NaN','NA', 'None', '--None--', 'NaN'] #Add unknown values present in the variables
column_values =  ["AF_LEAD_SOURCE__c", "AF_REG_SOURCE__c"] #Add columns for which null values have to be re-placed

percentage = []
missing_values = []
features = []

def missingvalues_check(): #checks for null values
    for i in range(0, len(column_values)):
        if filter_data[column_values[0]].isnull().values.any() == True:
            print column_values[i] + " has N/A values"
            missing_values.append(column_values[i])
        elif filter_data[column_values[0]].isnull().values.any() == False:
            print column_values[i] + " has no N/A values"
    

def NAcalculator(values): #checks if percentaqge of null values is less that 20% and creates a list of these features.
    for i in values:
        na = filter_data[i].replace(unknown_values, 'N/A')
        NAdf = pd.DataFrame(na.value_counts())
        NAdf1 = NAdf[i]
        if 'N/A' in NAdf1:
            perc = (int(NAdf1['N/A']) / float(NAdf1.sum()) * 100) 
            percentage.append((perc,i))  
            if perc < 20:
                features.append(i)
        else: 
            print column_values[i] + " has no N/A values"


            
def dummy_variables(values): #creates dummy varibales for feature list and returns adjusted R-square value against the o/p for each feature
    min_max_scaler = preprocessing.MinMaxScaler()
    minmax = min_max_scaler.fit_transform(filter_data["Total_Raised_Master"])
    #print len(values)
    for i in values:
        df = pd.DataFrame(filter_data[i])
        dummy_variable = pd.get_dummies(df)        
        for j in dummy_variable:
            variable = dummy_variable.loc[:,j]
            column_names = j
            x = np.asarray(variable)            
            y = np.asarray(minmax)    
            print column_names
            model = smf.OLS(y,x).fit()
            if "R-squared:" in str(model.summary()):
                print str(model.summary())[198:240]

In [17]:
missingvalues_check()
NAcalculator(missing_values)

AF_LEAD_SOURCE__c has N/A values
AF_REG_SOURCE__c has N/A values


In [18]:
# returns percentage of null values in the feature
NA_perc = pd.DataFrame(np.array(percentage), columns = ["Percentage","Features"])
NA_perc

,Percentage,Features
0,3.6268391442,AF_LEAD_SOURCE__c
1,14.4019096455,AF_REG_SOURCE__c


In [19]:
features

['AF_LEAD_SOURCE__c', 'AF_REG_SOURCE__c']

In [20]:
dummy_variables(features)

C:\Users\rvardhineni\AppData\Local\Continuum\Anaconda2\envs\py27\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\rvardhineni\AppData\Local\Continuum\Anaconda2\envs\py27\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


AF_LEAD_SOURCE__c_-1
R-squared:                       0.000
Mod
AF_LEAD_SOURCE__c_19
R-squared:                       0.000
Mod
AF_LEAD_SOURCE__c_BRC Add'l Name
R-squared:                       0.000
Mod
AF_LEAD_SOURCE__c_Bus or Train
R-squared:                       0.000
Mod
AF_LEAD_SOURCE__c_Bus or Train Location
R-squared:                       0.002
Mod
AF_LEAD_SOURCE__c_Community Event
R-squared:                       0.000
Mod
AF_LEAD_SOURCE__c_Community Involvement
R-squared:                       0.000
Mod
AF_LEAD_SOURCE__c_Direct Mail or Postcard
R-squared:                       0.000
Mod
AF_LEAD_SOURCE__c_Email
R-squared:                       0.056
Mod
AF_LEAD_SOURCE__c_Friend/Family/Acquaintance
R-squared:                       0.034
Mod
AF_LEAD_SOURCE__c_Internet
R-squared:                       0.037
Mod
AF_LEAD_SOURCE__c_Internet Search Engine
R-squared:                       0.000
Mod
AF_LEAD_SOURCE__c_Local Print
R-squared:                       0.000
Mod
AF_LEAD_SOUR